In [1]:
import sys

import test
sys.path.insert(0, "../hytek-parser")  # path to your forked parser folder
from hytek_parser import hy3_parser
from hytek_parser import export_xls

file_path = "../SampleMeetResults/Meet Results-PCS Spring Home Meet 2025-22Mar2025-002.hy3"

parsed_file = hy3_parser.parse_hy3(file_path)
events = parsed_file.meet.events
# print(events)
events_numbers = list((events.keys()))
for i in range(len(events_numbers)):
    event = events[events_numbers[i]]
    entries = event.entries
    for eventEntry in entries:
        print(eventEntry.swimmers[0].first_name, eventEntry.swimmers[0].last_name, eventEntry.swimmers[0].age, eventEntry.swimmers[0].date_of_birth)

    


Invalid line code: B3
Jeffrey Bzdill 20 1900-01-01
Evan Dabakarov 20 1900-01-01
Josh Dewey 20 1900-01-01
Colton Jiorle 20 1900-01-01
Jathan Krall 20 1900-01-01
Ian Chang 0 1900-01-01
Matthew Dramer 0 1900-01-01
Adam Drover 0 1900-01-01
Colton Park 0 1900-01-01
Vikraanth Sinha 0 1900-01-01
Abram Spinola 0 1900-01-01
Stefan Stratimirovic 0 1900-01-01
Zachary Sutterlin 0 1900-01-01
Trent Vacula 20 1900-01-01
Seth Cooper 20 2004-10-14
Giuseppe Porco 20 2004-06-28
Jeffrey Bzdill 20 1900-01-01
Evan Dabakarov 20 1900-01-01
Jathan Krall 20 1900-01-01
Michael Belch 0 1900-01-01
Daniel Chimes 0 1900-01-01
Jack Moriarty 0 1900-01-01
Colton Park 0 1900-01-01
Matvey Zoukovski 0 1900-01-01
Declan Rainowski 20 1900-01-01
Seth Cooper 20 2004-10-14
Maximus Forster 0 1900-01-01
Emre Oncel 20 1900-01-01
Luke Rutkowski 19 1900-01-01
Harrison Sly 20 1900-01-01
Evan Dabakarov 20 1900-01-01
Aiden Suzensky 0 1900-01-01
Stanley Zelaya 0 1900-01-01
Harrison Sly 20 1900-01-01
Josh Dewey 20 1900-01-01
Daniel Chim

c:\Users\chess\OneDrive\Documents\ISCAClient\ISCAPointScorer\.venv\Lib\site-packages\attr\_make.py:918: RuntimeWarning: Running interpreter doesn't sufficiently support code object introspection.  Some features like bare super() or accessing __class__ will not work with slotted classes.
  set_closure_cell(cell, cls)


In [ ]:
def get_event_codes(event):
    
    code = ""

    stroke = {
        Stroke.FREESTYLE: "Freestyle",
        Stroke.BACKSTROKE: "Backstroke",
        Stroke.BREASTSTROKE: "Breaststroke",
        Stroke.BUTTERFLY: "Butterfly",
        Stroke.MEDLEY: "Medley",
    }.get(event.stroke, "Unknown")
    
    if stroke == "Freestyle" and event.relay == False:
        code += "1"
    elif stroke == "Freestyle" and event.relay == True:
        code += "6"
    elif stroke == "Backstroke":
        code += "2"
    elif stroke == "Breaststroke":
        code += "3"
    elif stroke == "Butterfly":
        code += "4"
    elif stroke == "Medley" and event.relay == False:
        code += "5"
    elif stroke == "Medley" and event.relay == True:
        code += "7"
    else:
        return None        
    
    code += str(event.distance)

    if event.course == Course.SCY:
        code += "Y"
    elif event.course == Course.LCM:
        code += "L"
    elif event.course == Course.SCM:
        code += "S"
    else:
        return None

    return code            

In [ ]:
from collections import defaultdict
from hytek_parser.hy3.enums import (
    Course,
    DisqualificationCode,
    Gender,
    GenderAge,
    ResultType,
    Stroke,
    WithTimeTimeCode,
)


def get_event_name(event):
    gender = {
        Gender.MALE: "Men's",
        Gender.FEMALE: "Women's",
        Gender.UNKNOWN: "Mixed",
    }.get(event.gender, "Unknown")

    stroke = {
        Stroke.FREESTYLE: "Freestyle",
        Stroke.BACKSTROKE: "Backstroke",
        Stroke.BREASTSTROKE: "Breaststroke",
        Stroke.BUTTERFLY: "Butterfly",
        Stroke.MEDLEY: "Medley",
    }.get(event.stroke, "Unknown")

    return f"{gender} {event.distance} {stroke} ({event.course.name})" 

results = defaultdict()
for i in range(len(events_numbers)):
    event = events[events_numbers[i]]
    event_name = get_event_name(event)
    # print(event_name)
    entry = event.entries
    event_results = []
    for j in range(len(entry)):
        singleEntry = entry[j]
        swimmer_name = f"{singleEntry.swimmers[0].first_name} {singleEntry.swimmers[0].middle_initial + " " if singleEntry.swimmers[0].middle_initial else ''}{singleEntry.swimmers[0].last_name}"
        swimmer_age = singleEntry.swimmers[0].age
        swimmer_prelim_time = singleEntry.prelim_time
        swimmer_swimmoff_time = singleEntry.swimoff_time
        swimmer_final_time = singleEntry.finals_time
        resultEntry = {
            "swimmer": swimmer_name,
            "age" : swimmer_age,
            "prelim_time": swimmer_prelim_time,
            "swimoff_time": swimmer_swimmoff_time,
            "final_time": swimmer_final_time,
        }
        event_results.append(resultEntry)

    sorted_eventResults = sorted(event_results, key=lambda x: float('inf') if x['final_time'] == 0.0 or x['final_time'] is None else x['final_time'])
    results[event_name] = sorted_eventResults
    
# print(results)

In [ ]:
pointSystem15plus = {
    "Men's 100 Freestyle (SCY)": {
        41.23: 1000,
        43.56: 950,
        45.89: 900,
        46.99: 850,
        48.09: 800,
        49.19: 750,
        50.29: 700,
        51.34: 650,
        52.49: 600,
        53.56: 550,
        54.64: 500,
        55.71: 450,
        56.79: 400,
        57.89: 350,
        58.99: 300,
        60.09: 250,
        61.19: 200,
        62.19: 150,
        63.19: 100
    },
    "Women's 100 Freestyle (SCY)": {
        46.09: 1000,
        48.69: 950,
        51.29: 900,
        52.49: 850,
        53.69: 800,
        54.89: 750,
        56.09: 700,
        57.34: 650,
        58.59: 600,
        59.81: 550,
        61.04: 500,
        62.36: 450,
        63.49: 400,
        64.69: 350,
        65.89: 300,
        67.09: 250,
        68.29: 200,
        69.29: 150,
        70.29: 100
    }
}


In [ ]:
from scipy.interpolate import interp1d
import numpy as np


def get_interpolated_score(event_name: str, gender: str, time_seconds: float, points_dict: dict) -> float:
    """
    Interpolates a score from the pointSystem15plus dict based on event, gender, and performance time in seconds.
    
    Args:
        event_name (str): Event name, e.g., "100 Freestyle (SCY)"
        gender (str): 'Men' or 'Women'
        time_seconds (float): Performance time in total seconds
        points_dict (dict): Dictionary of times to scores
    
    Returns:
        float: Interpolated score
    """
    if time_seconds < 0:
        raise ValueError("Time cannot be negative")
    if time_seconds == 0:
        return 0.0
    key = f"{gender}'s {event_name}"
    if key not in points_dict:
        raise ValueError(f"No data found for event '{key}'")
    
    time_points = sorted(points_dict[key].items())  # list of (time, points)
    times = [t for t, _ in time_points]
    scores = [s for _, s in time_points]

    f = interp1d(times, scores, bounds_error=False, fill_value="extrapolate")
    score = float(f(time_seconds)) 
    return score if score > 0 else 0.0

In [ ]:

for swimmer_result in results["Women's 100 Freestyle (SCY)"]:
    swimmer_name = swimmer_result["swimmer"]
    swimmer_time = swimmer_result["final_time"]
    if swimmer_time is not None or swimmer_time != 0.0:
        # Convert time to total seconds
        minutes, seconds = divmod(swimmer_time, 60)
        total_seconds = minutes * 60 + seconds
        # Get the interpolated score
        score = get_interpolated_score("100 Freestyle (SCY)", "Women", total_seconds, pointSystem15plus)
        print(f"Swimmer: {swimmer_name}, Time: {swimmer_time}, Score: {score}")

In [ ]:
for swimmer_result in results["Men's 100 Freestyle (SCY)"]:
    swimmer_name = swimmer_result["swimmer"]
    swimmer_time = swimmer_result["final_time"]
    if swimmer_time is not None or swimmer_time != 0.0:
        # Convert time to total seconds
        minutes, seconds = divmod(swimmer_time, 60)
        total_seconds = minutes * 60 + seconds
        # Get the interpolated score
        score = get_interpolated_score("100 Freestyle (SCY)", "Men", total_seconds, pointSystem15plus)
        print(f"Swimmer: {swimmer_name}, Time: {swimmer_time}, Score: {score}")